# Import libraries

In [ ]:
!pip install gdown==4.6.0

In [ ]:
import os
import gdown
import json
import pandas as pd
import urllib.request
from zipfile import ZipFile

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store data

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes', exist_ok=True)

# Download **TotalDefMeme**

Nirmalendu Prakash, Ming Shan Hee, and Roy Ka-Wei Lee. 2023. TotalDefMeme: A Multi-Attribute Meme dataset on Total Defence in Singapore. In Proceedings of the 14th Conference on ACM Multimedia Systems (MMSys '23). Association for Computing Machinery, New York, NY, USA, 369–375. https://doi.org/10.1145/3587819.3592545

In [ ]:
# Download large file from Google Drive. via
# https://github.com/wkentaro/gdown
url = 'https://drive.google.com/u/0/uc?id=1oJIh4QQS3Idff2g6bZORstS5uBROjUUz'
output = '/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes.zip'
gdown.download(url, output, quiet=False)

In [ ]:
# Unzip file. via
# https://www.geeksforgeeks.org/unzipping-files-in-python/
with ZipFile('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes.zip',
             'r') as zObject:
    zObject.extractall('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/')

In [ ]:
os.remove('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes.zip')

In [ ]:
urllib.request.urlretrieve(
    'https://raw.githubusercontent.com/Social-AI-Studio/Total-Defense-Memes/main/report/annotation.json',
    '/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes.json')

# Load **TotalDefMeme Dataset**

In [ ]:
data = json.load(open('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes.json'))
# Build dataframe from list of dictionaries. via
# https://stackoverflow.com/a/72327549
total_defense_memes = pd.DataFrame.from_dict({k: v for d in data['Pillar_Stances'] for k,v in d.items()}, orient='index')
total_defense_memes = total_defense_memes.rename(columns={0: 'first', 1: 'second', 2: 'third'})
total_defense_memes['image'] = total_defense_memes.index
total_defense_memes = total_defense_memes.reset_index(drop=True)

# Reshape dataset

In [ ]:
total_defense_memes = pd.melt(total_defense_memes, id_vars='image', value_vars=['first', 'second', 'third'], value_name='pillars_and_stances')
total_defense_memes = total_defense_memes.drop('variable', axis=1)
total_defense_memes = total_defense_memes.dropna().reset_index(drop=True)
total_defense_memes[['pillar','stance_list']] = total_defense_memes['pillars_and_stances'].apply(pd.Series)
total_defense_memes = total_defense_memes[total_defense_memes['pillar'].str.contains('Others') == False]
total_defense_memes = total_defense_memes.reset_index(drop=True)
total_defense_memes['stance_list'] = total_defense_memes['stance_list'].apply(sorted)

# Separately subset rows where all annotators agree, and where the majority agree

In [ ]:
# Filter rows where list-column contains a specific set of items. via
# https://stackoverflow.com/a/63094115
total_defense_memes['joined'] = total_defense_memes['stance_list'].str.join(sep=',')
total_defense_memes_test = total_defense_memes[(total_defense_memes['joined'].isin(['Neutral,Neutral'])) | \
    (total_defense_memes['joined'].isin(['Neutral,Neutral,Neutral'])) | \
    (total_defense_memes['joined'].isin(['Supportive,Supportive'])) | \
    (total_defense_memes['joined'].isin(['Supportive,Supportive,Supportive'])) | \
    (total_defense_memes['joined'].isin(['Against,Against'])) | \
    (total_defense_memes['joined'].isin(['Against,Against,Against']))]
total_defense_memes_test.insert(5, 'stance', total_defense_memes_test['stance_list'].apply(lambda x: x[0]))
total_defense_memes_test = total_defense_memes_test.drop('joined', axis=1)
total_defense_memes_test = total_defense_memes_test.reset_index(drop=True)

total_defense_memes_examples = total_defense_memes[(total_defense_memes['joined'].isin(['Against,Neutral,Neutral'])) | \
    (total_defense_memes['joined'].isin(['Against,Against,Neutral'])) | \
    (total_defense_memes['joined'].isin(['Neutral,Neutral,Supportive'])) | \
    (total_defense_memes['joined'].isin(['Neutral,Supportive,Supportive'])) | \
    (total_defense_memes['joined'].isin(['Against,Against,Supportive'])) | \
    (total_defense_memes['joined'].isin(['Against,Supportive,Supportive']))]
# Find majority element in list. via
# https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/
total_defense_memes_examples.insert(5, 'stance', total_defense_memes_examples['stance_list'].apply(lambda x: max(set(x), key = x.count)))
total_defense_memes_examples = total_defense_memes_examples.drop('joined', axis=1)
dup = list(set(total_defense_memes_test['image'].values).intersection(total_defense_memes_examples['image'].values))
total_defense_memes_examples = total_defense_memes_examples[~total_defense_memes_examples['image'].isin(dup)]
total_defense_memes_examples = total_defense_memes_examples.reset_index(drop=True)

# Prefix image names with image directories

In [ ]:
total_defense_memes_test['image'] = '/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/TD_Memes/' + total_defense_memes_test['image'].astype(str)
total_defense_memes_examples['image'] = '/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/TD_Memes/' + total_defense_memes_examples['image'].astype(str)

# Save datasets in CSV format

In [ ]:
total_defense_memes_test.to_csv('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes_test.csv', index=False)
total_defense_memes_examples.to_csv('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes_examples.csv', index=False)